<a href="https://colab.research.google.com/github/murillo-borges/webscraping-livelo-esfera/blob/main/Webscraping_Whats_Milhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dados padronizados

In [8]:
pip install pandas

In [9]:
pip install datetime

### Horarios envios
- 08h
- 11h
- 14h
- 18h

In [10]:
#Horarios de corte para envios
hora_max_grupo_perfumaria_cosmeticos = 9
hora_min_grupo_esportivo = 11
hora_max_grupo_esportivo = 13
hora_min_grupo_cosmeticos = 14
hora_max_grupo_cosmeticos = 18
hora_min_grupo_varejo = 17
hora_max_grupo_varejo = 19

### Função para Webscrapping

In [11]:
#Importando bibliotecas
import requests
import pandas as pd
import re
import datetime
from bs4 import BeautifulSoup
from zoneinfo import ZoneInfo
import textwrap

#Id dos grupos
id_grupo_envio = '555195564436-1477420627@g.us'
  #Meu grupo pessoal = '555195564436-1477420627@g.us'
  #Grupo milhas business = '120363040547851644@g.us'

#Coletando a data do dia
data_hoje = datetime.datetime.now(ZoneInfo("America/Sao_Paulo")).date()
data_formatada = data_hoje.strftime("%d/%m/%Y")

# Lista de nomes que você quer capturar (normalizados)
nomes_alvo = ["livelo", "shopping livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "pass", "LATAM Pass", "Shopping Livelo"]

#Buscando a hora atual
agora_brasilia = datetime.datetime.now(ZoneInfo("America/Sao_Paulo"))

hora_atual = int(agora_brasilia.strftime("%H"))

import requests, re
import pandas as pd
from bs4 import BeautifulSoup

def coletar_promocoes(url, parceiro, data_formatada, nomes_alvo):
    pagina = requests.get(url)
    dados_pagina = BeautifulSoup(pagina.text, 'html.parser')

    # --- Captura dos nomes ---
    list_nomes_clubes = []
    for sp in dados_pagina.select("div.d-grid span"):
        nome = sp.get_text(strip=True)
        nome_norm = nome.lower()

        if nome_norm not in nomes_alvo:
            continue

        # Emojis
        if nome_norm in("livelo", "Shopping Livelo"):
            nome = f"🟣 {nome}"
        elif nome_norm == "esfera":
            nome = f"🔴 {nome}"
        elif nome_norm == "smiles":
            nome = f"🟠 {nome}"
        elif nome_norm in ("azul", "shopping latam", "latam"):
            nome = f"🔵 {nome}"
        else:
            nome = nome

        list_nomes_clubes.append(nome)

    # --- Captura da pontuação ---
    list_pontuacao_clubes = []
    tabela_pm = dados_pagina.find('th', string=lambda s: s and 'Pontos e Milhas' in s)
    if tabela_pm:
        table = tabela_pm.find_parent('table')
        for tr in table.select('tbody tr'):
            nome_el = tr.select_one('td .d-grid span')
            if not nome_el:
                continue
            nome_norm = nome_el.get_text(strip=True).lower()

            if nome_norm not in nomes_alvo:
                continue

            tds = tr.find_all('td')
            if len(tds) < 2:
                continue

            texto_ganho = tds[-1].get_text(" ", strip=True)
            m = re.search(r'(\d+(?:[.,]\d+)?)\s*pt', texto_ganho, flags=re.I)
            if not m:
                continue

            valor = m.group(1).replace(',', '.')
            val_float = float(valor)
            val_final = int(val_float) if val_float.is_integer() else val_float

            list_pontuacao_clubes.append(val_final)

    # Monta DataFrame
    df = pd.DataFrame({
        "nome_clube": list_nomes_clubes,
        "pontuacao_clube": list_pontuacao_clubes
    })

    # --- Ajuste de pontuação e análise ---
    list_analise = []
    for index, row in df.iterrows():
        nome_norm = row["nome_clube"].lower()

        if ("livelo" not in nome_norm) and ("esfera" not in nome_norm):
            pontuacao_ajustada = row["pontuacao_clube"] / 1.8
        else:
            pontuacao_ajustada = row["pontuacao_clube"]

        #Analisando a promoção
        if pontuacao_ajustada >= 12:
          d_analise = '⭐⭐⭐⭐⭐ Nível 5 (Excelente / Raro) 😏: Promoção rara, aproveite sem medo!'
        elif pontuacao_ajustada >= 8:
          d_analise = '⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, daqui pra cima já vale muito!'
        elif pontuacao_ajustada >= 6:
          d_analise = '⭐⭐⭐ Nível 3 (Bom) 😉: Bom momento para potencializar compras planejadas, mas se puder aguardar, tem coisa melhor!'
        elif pontuacao_ajustada >= 3:
          d_analise = '⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você realmente já iria comprar, mas é melhor aguardar algo melhor ;)'
        else:
          d_analise = '⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor'

        list_analise.append(d_analise)

    df["analise_pontuacao"] = list_analise
    df["parceiro"] = parceiro
    df["data_coleta_promocao"] = data_formatada

    return df

### Mensagem Bom dia

In [12]:
#Criando o texto de bom dia

dia_semana = agora_brasilia.strftime("%A")  # Nome do dia em inglês

if dia_semana == "Monday":
    txt_dia_semana = "segundou! 😴"
elif dia_semana == "Tuesday":
    txt_dia_semana = "terçou!"
elif dia_semana == "Wednesday":
    txt_dia_semana = "quartou!"
elif dia_semana == "Thursday":
    txt_dia_semana = "quintou! 🚀"
elif dia_semana == "Friday":
    txt_dia_semana = "sextou! 🔥"
elif dia_semana == "Saturday":
    txt_dia_semana = "sabadou! 😎"
elif dia_semana == "Sunday":
    txt_dia_semana = "domingou! ☀️"
else:
    txt_dia_semana = "vocês estão bem?"

texto_bom_dia = f"""
☀️ *Bom dia milheiros, {txt_dia_semana}*

Antes de começar o dia, segue o nosso critério de análise para você aproveitar as melhores promoções no acúmulo de milhas:

Nós avaliamos cada promoção em 5 níveis, com base no retorno de milhas/pontos:

• *Nível 1 (Ruim) ⭐* Menos que 3 pontos → Sugiro aguardar algo melhor.
• *Nível 2 (Mediano) ⭐⭐* 3 pontos ou mais → Use só se você já iria comprar de qualquer forma.
• *Nível 3 (Bom) ⭐⭐⭐* 6 pontos ou mais → Bom para compras planejadas, mas ainda pode melhorar.
• *Nível 4 (Muito Bom) ⭐⭐⭐⭐* 8 pontos ou mais → Ótima promoção, daqui pra cima já vale muito a pena!
• *Nível 5 (Excelente / Raro) ⭐⭐⭐⭐⭐* 12 pontos ou mais → Promoção rara, dessas que não aparecem toda hora. Aproveite sem medo!

Quanto maior o nível, melhor o custo-benefício para acumular milhas/pontos.

Obs: Em promoções diretamente nas cias aéreas, nos aplicamos uma divisão de 1,80 para igualar as milhas ao pontos (considerando
uma transferência bonificada de 80%)!

Se você está gostando das nossas análises, deixe o seu like nessa mensagem ;)

Ótimo dia a todos! Bora pra cima! 🚀

Att @murilloinvestor
"""

print(texto_bom_dia)


☀️ *Bom dia milheiros, quintou! 🚀* ✈️

Antes de começar o dia, segue o nosso critério de análise para você aproveitar as melhores promoções no acúmulo de milhas:

Nós avaliamos cada promoção em 5 níveis, com base no retorno de milhas/pontos:

• *Nível 1 (Ruim) ⭐* Menos que 3 pontos → Sugiro aguardar algo melhor.
• *Nível 2 (Mediano) ⭐⭐* 3 pontos ou mais → Use só se você já iria comprar de qualquer forma.
• *Nível 3 (Bom) ⭐⭐⭐* → 6 pontos ou mais Bom para compras planejadas, mas ainda pode melhorar.
• *Nível 4 (Muito Bom) ⭐⭐⭐⭐* 8 pontos ou mais → Ótima promoção. Daqui pra cima já vale muito a pena!
• *Nível 5 (Excelente / Raro) ⭐⭐⭐⭐⭐* 12 pontos ou mais → Promoção rara, dessas que não aparecem toda hora. Aproveite sem medo!

Quanto maior o nível, melhor o custo-benefício para acumular milhas/pontos.

Obs: Em promoções diretamente nas cias aéreas, nos aplicamos uma divisão de 1,80 para igualar as milhas ao pontos (considerando
uma transferência bonificada de 80%)!

Se você está gostando d

## Grupo de Perfumaria (08h am) GRUPO OK

### Loja Natura

In [13]:
hora_atual = agora_brasilia.hour

if hora_atual < hora_max_grupo_perfumaria_cosmeticos:
    df_natura = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-natura/",
        parceiro="Natura",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_natura)
else:
    print("Fora do horário!")

nome_clube  pontuacao_clube  \
0          🔴 Esfera              2.0   
1          🟠 Smiles              4.5   
2            🔵 Azul              4.0   
3          🟣 Livelo              4.0   
4  🔵 Shopping Latam              3.0   

                                   analise_pontuacao parceiro  \
0    ⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor   Natura   
1    ⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor   Natura   
2    ⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor   Natura   
3  ⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você re...   Natura   
4    ⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor   Natura   

  data_coleta_promocao  
0           30/10/2025  
1           30/10/2025  
2           30/10/2025  
3           30/10/2025  
4           30/10/2025

### Loja Boticário

In [14]:
hora_atual = agora_brasilia.hour

if hora_atual < hora_max_grupo_perfumaria_cosmeticos:
    df_boticario = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-boticario/",
        parceiro="Boticário",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_boticario)
else:
    print("Fora do horário!")

nome_clube  pontuacao_clube  \
0   🟣 Livelo                6   
1   🔴 Esfera                2   

                                   analise_pontuacao   parceiro  \
0  ⭐⭐⭐ Nível 3 (Bom) 😉: Bom momento para potencia...  Boticario   
1    ⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor  Boticario   

  data_coleta_promocao  
0           30/10/2025  
1           30/10/2025

### Loja Eudora

In [15]:
hora_atual = agora_brasilia.hour

if hora_atual < hora_max_grupo_perfumaria_cosmeticos:
    df_eudora = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-eudora/",
        parceiro="Eudora",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_eudora)
else:
    print("Fora do horário!")

nome_clube  pontuacao_clube  \
0   🟣 Livelo                2   
1   🔴 Esfera                1   

                                 analise_pontuacao parceiro  \
0  ⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor   Eudora   
1  ⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor   Eudora   

  data_coleta_promocao  
0           30/10/2025  
1           30/10/2025

### Enviando os dados via Whatsapp

In [16]:
if hora_atual < hora_max_grupo_perfumaria_cosmeticos:
  # Concatenando os dataframes
  df_final = pd.concat([df_natura, df_boticario, df_eudora], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"💄 *Resumo das Promoções da Categoria de Perfumaria e Cosméticos em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += textwrap.dedent(f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}""").strip()

  msg_grupo = msg_titulo + "\n""\n" + msg


  # Enviando mensagem no grupo do Whatsapp
  import requests

  # Substitua com os dados reais
  INSTANCE_ID = 'inst_4ng9578nwiz0'
  API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

  # JID do grupo (obtido via rota /groups)
  group_jid = '120363040547851644@g.us'

  # Mensagem de Bom dia
  message_bom_dia = texto_bom_dia
  message_promo = msg_grupo

  # Endpoint
  url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/text"

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  # Payload
  payload = {
      "jid": group_jid,
      "message": message_bom_dia,
  }

  #Enviando a mensagem de bom dia
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem de bom dia enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")


  # Aguardar 2 minutos
  import time
  time.sleep(120)

  # Payload
  payload = {
      "jid": group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")

Mensagem de bom dia enviada com sucesso para o grupo!
Mensagem com as promoções enviada com sucesso para o grupo!


##

## Grupo de Roupas e Calçados Esportivos (11h am) GRUPO OK

### Loja Centauro

In [17]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_centauro = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-centauro/",
        parceiro="Centauro",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_centauro)
else:
    print("Fora do horário!")

Fora do horário!


### Loja NetShoes

In [18]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_netshoes = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-netshoes/",
        parceiro="Netshoes",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_netshoes)
else:
    print("Fora do horário!")

Fora do horário!


### Loja New Balance

In [19]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_new_balance = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-new-balance/",
        parceiro="New Balance",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_new_balance)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Nike

In [20]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_nike = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-nike/",
        parceiro="Nike",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_nike)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Adidas

In [21]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_adidas = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-adidas/",
        parceiro="Adidas",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_adidas)
else:
    print("Fora do horário!")

Fora do horário!


### Enviando os dados via Whatsapp

In [22]:
if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
  # Concatenando os dataframes
  df_final = pd.concat([df_centauro, df_netshoes, df_new_balance, df_nike, df_adidas], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"👟🏓 *Promoções da Categoria de Roupas e Calçados Esportivos em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += textwrap.dedent(f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}""").strip()

  msg_grupo = msg_titulo + "\n\n" + msg

  # Enviando mensagem no grupo do Whatsapp
  import requests

  # Substitua com os dados reais
  INSTANCE_ID = 'inst_4ng9578nwiz0'
  API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

  # JID do grupo (obtido via rota /groups)
  group_jid = '120363040547851644@g.us'

  # Mensagem
  message_promo = msg_grupo

  # Endpoint
  url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/text"

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  # Payload
  payload = {
      "jid": group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")


Fora do horário!


## Grupo de Beleza e Cosméticos (14h am)

### Loja Época Cosméticos

In [23]:
nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_cosmeticos and hora_atual < hora_max_grupo_cosmeticos:
    df_epoca_cosmeticos = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-epoca-cosmeticos/",
        parceiro="Época Cosméticos",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_epoca_cosmeticos)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Sephora

In [24]:
nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_cosmeticos and hora_atual < hora_max_grupo_cosmeticos:
    df_sephora = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-sephora/",
        parceiro="Sephora",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_sephora)
else:
    print("Fora do horário!")

Fora do horário!


### Enviando os dados via Whatsapp

In [25]:
if hora_atual >= hora_min_grupo_cosmeticos and hora_atual < hora_max_grupo_cosmeticos:
  # Concatenando os dataframes
  df_final = pd.concat([df_epoca_cosmeticos, df_sephora], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"💄 *Resumo das Promoções da Categoria de Beleza e Cosméticos em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}"""

  msg_grupo = msg_titulo + "\n""\n" + msg

  # Enviando mensagem no grupo do Whatsapp
  import requests

  # Substitua com os dados reais
  INSTANCE_ID = 'inst_4ng9578nwiz0'
  API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

  # JID do grupo (obtido via rota /groups)
  group_jid = id_grupo_envio

  # Mensagem
  message_promo = msg_grupo

  # Endpoint
  url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/text"

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  # Payload
  payload = {
      "jid": group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")


Fora do horário!


## Grupo de Varejo (17h am) GRUPO OK

### Loja Amazon (Latam pass nao esta aparecendo)

In [26]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_amazon = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-amazon/",
        parceiro="Amazon",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_amazon)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Ponto

In [27]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_ponto = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-ponto/",
        parceiro="Ponto",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_ponto)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Casas Bahia

In [28]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_casas_bahia = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-casas-bahia/",
        parceiro="Casas Bahia",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_casas_bahia)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Extra

In [29]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_extra = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-extra/",
        parceiro="Extra",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_extra)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Mercado Livre

In [30]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_mercado_livre = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-mercado-livre/",
        parceiro="Mercado Livre",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_mercado_livre)
else:
    print("Fora do horário!")

Fora do horário!


### Loja FastShop

In [31]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_fast_shop = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-fast-shop/",
        parceiro="Fast Shop",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_fast_shop)
else:
    print("Fora do horário!")

Fora do horário!


### Enviando os dados via Whatsapp

In [32]:
if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
  # Concatenando os dataframes
  df_final = pd.concat([df_amazon, df_mercado_livre, df_ponto, df_casas_bahia, df_extra, df_fast_shop], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"📱💻 *Promoções do Grupo de Varejo em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += textwrap.dedent(f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}""").strip()

  msg_grupo = msg_titulo + "\n""\n" + msg

  # Enviando mensagem no grupo do Whatsapp
  import requests

  # Substitua com os dados reais
  INSTANCE_ID = 'inst_4ng9578nwiz0'
  API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

  # JID do grupo (obtido via rota /groups)
  group_jid = '120363040547851644@g.us'

  # Mensagem
  message_promo = msg_grupo

  # Endpoint
  url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/text"

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  # Payload
  payload = {
      "jid": group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")


Fora do horário!


## ADD RENNER, C&A, RIACHUELO

## Jabares (Mensagens com Imagens)

### Jabar banco inter (11h am) GRUPO OK

In [33]:
if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo and dia_semana in ("Wednesday", "Saturday"):

  import requests

  # Aguardar 2 minutos
  import time
  time.sleep(120)

  # Substitua com os dados reais
  INSTANCE_ID = 'inst_4ng9578nwiz0'
  API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

  # JID do grupo (obtido via rota /groups)
  group_jid = '120363040547851644@g.us'

  import textwrap

  if dia_semana == "Wednesday":
    message_promo = textwrap.dedent("""
    *Quer começar no mundo dos cartões com o pé direito?*

    O Inter Prime é uma ótima escolha: pontua bem, os pontos não expiram e você ainda
    ganha acesso às Salas VIP de forma gratuita, perfeito para dar um upgrade nas suas viagens.

    Lembrando que a forma mais fácil de conseguir esse cartão é *assinando o plano Duo Gourmet anual* 👇

    💰 *Quero economizar R$30 no plano anual do Duo Gourmet?*

    Utilize o código B1EE1054 para ganhar R$ 30 off na assinatura do plano anual pelo app do Inter!
    👉 Acesse: https://intergo.app/1f148152

    🟠 *Abra a sua conta no Banco Inter com o nosso link e ganhe 200 pontos*

    Utilize o código 0F01D993 e abra a sua conta gratuita no Inter. Ao fazer isso, você já receberá 200 pontos Loop de presente!
    👉 https://intergo.app/242d85c4

    -----------------------------

    💳 Não conhece esse cartão? Assista o vídeo abaixo para conhecer os principais benefícios do cartão Inter Prime 👇
    https://www.youtube.com/playlist?list=PLywCMnijM298ruCE-Cey18WvsoV2UqqQf""").strip()

  else:
    message_promo = textwrap.dedent("""
    *Pensando em começar no mundo dos cartões? 💳*

    O Inter Prime é uma excelente opção: boa pontuação, pontos que não expiram e acesso às Salas VIP ✈️🔥

    A maneira mais simples de conseguir esse cartão é assinando o plano Duo Gourmet anual 👇

    💰 *Quer economizar R$30 no plano anual do Duo Gourmet?*
    Use o código **B1EE1054** no app do Inter
    👉 https://intergo.app/1f148152

    🟠 *Ainda não tem conta? abra sua conta no Banco Inter e ganhe 200 pontos Loop*
    Use o código **0F01D993**
    👉 https://intergo.app/242d85c4

    -----------------------------

    💳 Conheça os benefícios do cartão Inter Prime:
    https://www.youtube.com/playlist?list=PLywCMnijM298ruCE-Cey18WvsoV2UqqQf""").strip()

  # Imagem do Google Drive
  drive_id = "1OpOK2Ee1q2yJO-UDhfP3w3hFh2uM7y0s"
  img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

  url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/media"

  payload = {
      "jid": group_jid,
      "mediaType": "image",
      "mimetype": "image/jpeg",
      "media": img_url,
      "caption": message_promo,  # ← usa seu texto de análise
      "filename": "analise.jpg"
  }

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  response = requests.post(url, json=payload, headers=headers)

  print(response.json())

else:
  print("Fora do horário!")

Fora do horário!


### Jabar planilha (11h am) GRUPO OK

In [34]:
if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo and dia_semana in ("Thursday", "Sunday"):

  import requests

  # Aguardar 2 minutos
  import time
  time.sleep(120)

  # Substitua com os dados reais
  INSTANCE_ID = 'inst_4ng9578nwiz0'
  API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

  # JID do grupo (obtido via rota /groups)
  group_jid = '120363040547851644@g.us'

  import textwrap

  if dia_semana == "Thursday":
    message_promo = textwrap.dedent("""
    *Quer organizar suas finanças de um jeito simples e inteligente?*

    A nossa Planilha de Controle Financeiro foi feita para quem quer parar de gastar no automático
    e finalmente entender para onde o dinheiro está indo, de forma clara, rápida e sem complicação.

    Por que essa planilha é a solução perfeita para você?
    • Simples de usar e totalmente automatizada
    • Visão completa do seu fluxo de caixa
    • Dashboard visual e intuitivo

    💡 *E o melhor?*
    • Ela custa menos do que uma pizza 🍕
    • Você paga *uma única vez* e fica com a planilha *para sempre*.
    • E você ainda ganha uma aula passo a passo. Mesmo sem saber Excel, você vai conseguir usar sem dificuldade. 🎓✅
    • Ao adquirir, você ainda apoia o nosso trabalho e nos ajuda a continuar trazendo conteúdo gratuito aqui no grupo. 🙏

    👉 Assista ao vídeo e adquira sua planilha aqui:
    https://bit.ly/whatsplancontrolefinanceiro

    -----------------------------

    🧠 *Organização financeira muda tudo.*
    Comece hoje, seu “eu do futuro” vai agradecer.""").strip()
  else:
    message_promo = textwrap.dedent("""
    *Quer organizar suas finanças de um jeito simples?*

    Nossa Planilha de Controle Financeiro te ajuda a ver para onde seu dinheiro está indo e a controlar seus gastos sem complicação.

    • Custa menos que uma pizza 🍕
    • Pagou uma vez, é sua para sempre.
    • E tem aula passo a passo (mesmo se você não souber Excel). 🎓✅

    👉 Conheça e adquira aqui:
    https://bit.ly/whatsplancontrolefinanceiro""").strip()


  # Imagem do Google Drive
  drive_id = "1dHHTOXW8g3bZ4AGZk14Lvnu9mGSpHoVc"
  img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

  url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/media"

  payload = {
      "jid": group_jid,
      "mediaType": "image",
      "mimetype": "image/jpeg",
      "media": img_url,
      "caption": message_promo,  # ← usa seu texto de análise
      "filename": "analise.jpg"
  }

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  response = requests.post(url, json=payload, headers=headers)

  print(response.json())

else:
  print("Fora do horário!")

Fora do horário!


### Jabar cartao Ruby Stell

### Jabar cartao RecargaPay